In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
batch_size=4
block_size=8
max_iters=10000
eval_iters=250
learning_rate=3e-4
dropout=0.2
n_embed=384
n_layer=4
n_head=4
dropout=0.2
device= 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
with open("wizard_of_oz.txt",'r',encoding="utf-8") as f:
    text=f.read()

In [ ]:
chars=sorted(set(text))

In [ ]:
print(chars)
vocab_size=len(chars)

In [ ]:
string_to_int={ch:i for i,ch in enumerate(chars)}
int_to_string={i:ch for i,ch in enumerate(chars)}
encode= lambda s: [string_to_int[c] for c in s]
decode= lambda l:"".join([int_to_string[i] for i in l])

encode_hello=encode("Hello")
print(encode_hello)
print(decode(encode_hello))

In [ ]:
#make our data element or text data to torch tensors
data=torch.tensor(encode(text), dtype=torch.long)

In [ ]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [ ]:
block_size=8

x=train_data[:block_size]
y=train_data[1:block_size+1]

for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    
    print(f"when the input is {context}, the target is {target}")

In [ ]:
def get_batch(split):
    data= train_data if split=="train" else val_data
    ix=torch.randint(len(data)- block_size, (batch_size,))
    #print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

x,y =get_batch("train")
print("Inputs")
print(x)
print("outputs")
print(y)

In [ ]:
@torch.no_grad()#does not use gradient at all yet
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses=torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y=get_batch(split)
            logits, loss=model(X,Y)
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key=nn.Linear(n_embed,head_size, bins=False)
        self.query=nn.Linear(n_embed, head_size, bins=False)
        seld.values= nn.Linear(n_embed, head_size, bins=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout=nn.Dropout(dropout)
        
    def forward(self, x):
        B,T,C= x.shape
        k=self.key(x)
        q=self.query(x)
        #Compute Attetion 
        #Transposing is going to make sure 
        wei= Q@ k.transpose(-2,-1)* k.shape[-1]**-0.5
        wei=wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))
        wei=F.softmax(wei, dropout(wei))
        wei=self.dropout(wei)            
        v=self.value(x)
        out=wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj=nn.Linear(head_size * num_heads, n_embed)
        self.dropout=nn.Dropout(dropout)
    
    def forward(self, x):
        out=torch.cat([h(x) for h in self.heads], dims=-1) #(b,t,f)==>(B,t,[h1,h1,h1,h1,h2,h2,h2,h2,h3,h3,h3,h3])
        out= self.dropout(self.projection(out))
        return out
        

class FeedForward(nn.Module):
    def __init__( self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4*e_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
            )
    
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):#Every decoder is a block of the below implementation
    def __init__(seld, n_embed, n_head):
        super().__init__()
        head_size=n_embed//n_head
        self.sa=MultiHeadAttention(n_head, head_size)
        self.ffwd= FeedForward(n_embed)
        self.ln1=nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
    def forward(self, x):
        y= self.sa(x)
        x=self.ln1(x+y)
        y=self.ffwd(x)
        x=self.ln2(x+y)
        return x
    
        
class GPTLanguageModel(nn.Module):
    def __init__ (self, vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size, n_embed) #We need a large vector to store information which will be done by n_embed
        self.position_embedding_table= nn.Embedding(block_size, n_embed)# 8 different indices of n_embed size
        self.blocks=nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range n_layer])#Makes 4 layers of Decoder Sequentially
        
        self.in_f=nn.LayerNorm(n_embed)
        self.im_head=nn.Linear(n_embed,vocab_size) #Vocab size probab ditribution of the vocabulary
        
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(modlue, nn.Linear):
            torch.nn.init.normal_(module.weiht, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        else isinstance(module,nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def forward(self, index, targets=None):#We write it from scratch so that we know exactly what's going on and for flexibility
        logits=self.token_embedding_table(index)
        
        tok_emb= self.token_embedding_table(idx) #(B,t,c)
        pos_emb=self.position_embedding_table(torch.arange(T, device=device)) #(t,c)
        x=tok_emb + pos_emb#(b,t,c)
        x=self.blocks(x) #(b,t,c)
        x=self.in_f(x) #(b,t,c)
        logits= self.im_head(x) #(b,t,vocab_size) #We get to softmax and get probabs 
        
        if targets is None:
            loss=None
        else:
            B, T, C =logits.shape#Batch, Time, Dimension
            logits=logits.view(B*T,C)#Because we are paying attention to chanels of vocab
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits, targets)#cross-entropy expects the shape to be in B*T

        return logits,loss

    def generate(self, index, max_new_tokens):
        #index is (B,t) array of indices in the current context
        for _ in range(max_new_tokens):
            #Get the predictions
            logits, loss=self.forward(index)
            
            
            
            #focus only on the last time step
            logits=logits[:,-1,:] #Becomes (B,C)
            #apply softmax to get probabilities
            probs=F.softmax(logits, dim=-1)#we are focusing on the last dimension
            #sample from the distribution
            index_next=torch.multinomial(probs, num_samples=1)# (B,1)
            #append sampled index to the running sequence
            index=torch.cat((index, index_next),dim=1)#(B,T+1)
            
        return index

model=GPTLanguageModel(vocab_size)
m=model.to(device)

context=torch.zeros((1,1), dtype=torch.long,device=device)
generated_chars=decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(generated_chars)
print(len(generated_chars))
    